# MODELO IDENTIFICADOR DE ATRIBUTOS EN IMAGENES
Lázaro R. Díaz Lievano.

En este notebook se define y entrena un modelo para identificar atributos en imagenes como parte del curso de Redes Neuronales de la FCFM de la BUAP, se utiliza la base de datos de CelebA, la cual se puede encontrar en kaggle.

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers, models
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Dropout,Activation,MaxPooling2D,Flatten
from tensorflow.keras.optimizers import RMSprop, SGD



## Cargando los datos: lista de atributos e imagenes.

La lista de atributos inicialmente es un archivo de texto lleno de datos en forma de tabla, donde -1 se refiere a Falso y 1 a Verdadero, trabajar con este tipo de dato en este formato es tedioso por lo que lo mejor es convertirlo a un CSV, es decir, un archivo separado por comas, para ello reemplazamos los espacios por comas y podemos omitir trabajar con los encabezados que si bien para nosotros son importantes, para la red neuronal le es indiferente porque no tiene una idea de que signifique. 

In [4]:
#cargamos la lista de atributos
atributos = 'C:/Users/Lazaro Diaz/RNA_otono2022/Reconocimiento facial/list_attr_celeba.txt'
atributos_modificado = 'C:/Users/Lazaro Diaz/RNA_otono2022/Reconocimiento facial/list_attr_celeba_modificado.txt' 
#archivo en blanco donde se guardara la nueva lista

with open(atributos, 'r') as f:
    print("skipping: " + f.readline())
    print("skipping headers: " + f.readline())
    with open(atributos_modificado, 'w') as newf:
        for line in f:
            new_line = ' '.join(line.split())
            newf.write(new_line)
            newf.write('\n')
        

skipping: 202599

skipping headers: 5_o_Clock_Shadow Arched_Eyebrows Attractive Bags_Under_Eyes Bald Bangs Big_Lips Big_Nose Black_Hair Blond_Hair Blurry Brown_Hair Bushy_Eyebrows Chubby Double_Chin Eyeglasses Goatee Gray_Hair Heavy_Makeup High_Cheekbones Male Mouth_Slightly_Open Mustache Narrow_Eyes No_Beard Oval_Face Pale_Skin Pointy_Nose Receding_Hairline Rosy_Cheeks Sideburns Smiling Straight_Hair Wavy_Hair Wearing_Earrings Wearing_Hat Wearing_Lipstick Wearing_Necklace Wearing_Necktie Young 



Ahora definimos unos parametros de la red. Estos valores son arbitrarios, pero por cuestiones de tiempo es mejor iniciar con pocas epocas y asegurarnos que la red esté aprendiendo, conforme vayamos consiguiendo resultados aceptables podemos luego aumentar las epocas, por el contrario, si la red no aprende o es muy deficiente podemos aumentar el numero de batch_size, cambiar el optimizador, el learning rate o por supuesto, asegurarnos de que los datos con los que se entrena esten correctos.

In [5]:
epochs = 10
batch_size = 40
optimizer = 'rmsprop'
ih, iw = 192, 192 #tamaño de la imagen
input_shape = (ih, iw,3)


## Uniendo los dos tipos de dato en uno solo  

Una vez que ya terminamos con los datos de atributos y con las imagenes, podemos unirlas, cada lista de atributos con su correspondiente imagen, de esta forma la red asociará los atributos con la imagen. 

In [8]:
#Se define el dataframe 
atributos = 'C:/Users/Lazaro Diaz/RNA_otono2022/Reconocimiento facial/list_attr_celeba_modificado.txt'
df = pd.read_csv(atributos, sep=' ',  header=None)

#Se separan las imagenes y sus atributos para poder modificar los valores de -1 a 0, luego se vuelven a unir
files = tf.data.Dataset.from_tensor_slices(df[0])
attributes= tf.data.Dataset.from_tensor_slices(df.iloc[:,1:].to_numpy().astype('int64')).map(lambda x: ((x+1)/2))
data = tf.data.Dataset.zip((files,attributes))


ruta_imagenes = 'C:/Users/Lazaro Diaz/RNA_otono2022/Reconocimiento facial/img_align_celeba/'

def process_file(file_name, attributes):
    image = tf.io.read_file(ruta_imagenes + file_name)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [ih,iw])
    image /= 255.0
    return image, attributes

imagen_etiquetada = data.map(process_file).batch(batch_size)

In [10]:
#Se definen parámetros de la red y se dividen los datos en datos de entrenamiento y prueba

num_train = int(len(df)*0.7) #70% de los datos serán datos de prueba
num_test =len(df) - num_train #30% de los datos se usarán para evaluar la red.

epochs_steps = num_train // batch_size
test_steps = num_test // batch_size

data_train = imagen_etiquetada.take(num_train)
data_test = imagen_etiquetada.skip(num_train)


## Estructura de la red neuronal para identificar atributos

In [11]:
model = Sequential()

#Primera capa Convolucional
model.add(Conv2D(40, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Segunda capa Convolucional
model.add(Conv2D(80, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Tercera capa Convolucional
model.add(Conv2D(120, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Cuarta capa Plana o Densa
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))

#Capa de salida, aqui hay 40 neuronas correspondientes a cada atributo, 
#si la neurona se activa significa que la imagen posee el atributo
model.add(Dense(40))
model.add(Activation('sigmoid'))



In [12]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['binary_accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 190, 190, 40)      1120      
                                                                 
 activation (Activation)     (None, 190, 190, 40)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 95, 95, 40)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 93, 93, 80)        28880     
                                                                 
 activation_1 (Activation)   (None, 93, 93, 80)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 46, 46, 80)       0         
 2D)                                                    

## Entrenamiento del modelo  

In [14]:
history= model.fit(data_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=data_test,
    validation_steps=test_steps,
    #callbacks=[WandbCallback()
                   )


Epoch 1/10
5065/5065 [==============================] - 30227s 6s/step - loss: 0.2780 - binary_accuracy: 0.8796
Epoch 2/10
5065/5065 [==============================] - 8505s 2s/step - loss: 0.2466 - binary_accuracy: 0.8927
Epoch 3/10
5065/5065 [==============================] - 8326s 2s/step - loss: 0.2392 - binary_accuracy: 0.8960
Epoch 4/10
5065/5065 [==============================] - 8467s 2s/step - loss: 0.2351 - binary_accuracy: 0.8979
Epoch 5/10
5065/5065 [==============================] - 9207s 2s/step - loss: 0.2334 - binary_accuracy: 0.8988
Epoch 6/10
5065/5065 [==============================] - 8808s 2s/step - loss: 0.2340 - binary_accuracy: 0.8987
Epoch 7/10
5065/5065 [==============================] - 8989s 2s/step - loss: 0.2357 - binary_accuracy: 0.8981
Epoch 8/10
5065/5065 [==============================] - 9207s 2s/step - loss: 0.2384 - binary_accuracy: 0.8970
Epoch 9/10
5065/5065 [==============================] - 8948s 2s/step - loss: 0.2416 - binary_accuracy: 0.8956


In [15]:
model.save('rna_attrib.h5')